#### COMP90042 2023 Project: Automated Fact Checking For Climate Science Claims

The goal of this project is to design and implement a system that takes a given `claim` (which is a single sentence) and then retrieves (one or more) `evidence passages` (each passage is also a single sentence) from a document store, then using these evidence passages classifies the claim as one of these four labels: `[SUPPORTS, REFUTES, NOT_ENOUGH_INFO, DISPUTED]`.  

Our training data consists of a document store which is a set of $D$ evidence passages `{evidence-1, evidence-2, ..., evidence-D}` and training and validation data instances where each instance is a tuple of the form `(claim_text, claim_label, evidence_list)`, where evidence list is a subset of passages from the document store.

In this notebook, we explore the use of `BM25` for passage retreival and evaluate it's performance on the project dataset. Evaluation will involve comparing retreived evidence passages for each data instance with the ground truth evidence list  and computing precision, recall and F1 score. Then we will use the `average F1 score` across all data instances as the final evaluation metric.

In [2]:
from nltk.tokenize import RegexpTokenizer
from collections import defaultdict
from unidecode import unidecode
import math, random
from tqdm import tqdm
import json

#### Below is my implementation of a simple BM25 retreival system. Will later replace this with the PyLucene implementation.

In [ ]:
class IR_System():
    def __init__(self, k = 1.25, b = 0.75):
        self.k = k
        self.b = b
        self.tokenizer = RegexpTokenizer(r'\w+') 


    def train(self, documents):
        self.documents = documents
        self.TFIDF, self.inverted_index, self.doc_tfidf_norms = self.create_inverted_index()
        
    def tokenize(self, sent):
        # Replace accented letters with regular letters
        sent = unidecode(sent)
        # tokenize into words, remove punctuation
        return self.tokenizer.tokenize(sent.lower())

    def create_inverted_index(self):
        N = len(self.documents)
        TFIDF = defaultdict(float)
        inverted_index = defaultdict(list)

        # compute term frequency and document frequencies
        TF, term_docs = self.compute_TF_weighted()

        # create inverted index
        print(f"Computing TFIDF and creating inverted index...")
        for w, docs in tqdm(term_docs.items(), total=len(term_docs)):
            for d in sorted(list(docs)):
                tfidf = TF[(w,d)] * math.log10(N/len(docs))
                inverted_index[w].append(d)
                TFIDF[(w,d)] = tfidf

        # compute document TFIDF vector norms
        print(f"Computing TFIDF vector norms...")
        doc_tfidf_norms = [0] * N
        for d, doc in tqdm(enumerate(self.documents), total=len(self.documents)):
            words = self.tokenize(doc)
            for w in words:
                doc_tfidf_norms[d] = doc_tfidf_norms[d] +  TFIDF[(w,d)]**2
            doc_tfidf_norms[d] = math.sqrt(doc_tfidf_norms[d])

        return TFIDF, inverted_index, doc_tfidf_norms  
          
    # weighted TF for BM25
    def compute_TF_weighted(self):
        TF = defaultdict(int)
        term_docs = defaultdict(set)
        doc_length = defaultdict(float)
        Dtotal = 0
        print(f"Computing TFIDF...")
        for d, doc in tqdm(enumerate(self.documents), total=len(self.documents)):
            words = self.tokenize(doc)
            for w in words:
                TF[(w, d)] += 1
                term_docs[w].add(d)
            doc_length[d] = len(words)
            Dtotal += len(words)
        Davg = Dtotal / len(self.documents)

        # compute BM25 weighted term frequencies
        TF_weighted = defaultdict(float)
        for (w,d), tf in TF.items():
            TF_weighted[(w,d)] = (tf * (self.k + 1)) / (tf + self.k * (1 - self.b + self.b * (doc_length[d]/Davg)))
        return TF_weighted, term_docs
    

    def retrieve_docs(self, query, topk=1):
        query_words = self.tokenizer.tokenize(query.lower())
        #print(f"query words: {query_words}")
        # get all documents which contain words from query
        docs = []
        for w in query_words:
            docs.extend([d for d in self.inverted_index[w]])
        #print(f"docs: ")    
        # score all these documents
        scores = defaultdict(float)
        for d in docs:
            for w in query_words:
                scores[d] += self.TFIDF[(w,d)]
            scores[d] = scores[d] / self.doc_tfidf_norms[d]        
        #print(f"scores: {scores}")    
        # return topk documents
        sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        #print(f"sorted scores: {sorted_scores}")
        best = sorted_scores[:topk]
        topk_docs = []
        for doc, score in best:
            topk_docs.append((self.documents[doc], score))
        return topk_docs


#### Let's load up the data, both document store (`evidence.json`) and training/validation instances (`train-claims.json`, `dev-claims.json`).

In [10]:
# load the evidence passages
with open("project-data/evidence.json", "r") as train_file:
    document_store = json.load(train_file)         
print(f"Number of evidence passages: {len(document_store)}")

# load the training data insttances
with open("project-data/train-claims.json", "r") as train_file:
    train_data = json.load(train_file)
print(f"Number of training instances: {len(train_data)}")


# load the validation data instances
with open("project-data/dev-claims.json", "r") as dev_file:
    val_data = json.load(dev_file)    
print(f"Number of validation instances: {len(val_data)}")

Number of evidence passages: 1208827
Number of training instances: 1228
Number of validation instances: 154


1228